In [1]:
from dataloading import get_my_valid_dataset_unlimited, get_kfold_split_func, worker_init_fn
from torch.utils.data import IterableDataset, DataLoader

dset = get_my_valid_dataset_unlimited(split_method=get_kfold_split_func(total_k=5, current_k=0))
valid_loader = DataLoader(dset, batch_size=8, num_workers=16, pin_memory=True, persistent_workers=False, worker_init_fn=worker_init_fn)

valid files indicies
[3, 5, 6, 9, 32, 34, 42, 45, 48, 50, 61, 65, 66, 73, 78, 79, 82, 84, 95, 104, 106, 110, 113, 115, 128, 129, 131, 138, 139, 144, 151, 156, 159, 161, 177, 187, 188, 189, 199, 202, 205, 211, 219, 227, 232, 234, 235, 238, 239, 240, 252, 254, 257, 267, 268]
[2, 7, 8, 9, 10, 13, 17, 24, 32, 33, 36, 42, 44, 46, 48, 58, 64, 71, 74, 78, 86, 87, 88, 92, 93, 101, 103, 105, 117, 123, 137, 149, 159, 164, 165, 166, 173, 177, 180, 182, 185, 190, 191, 193, 198, 221, 228, 231, 233, 236, 241, 247, 251, 252, 256, 263, 264, 269, 276, 285, 295, 301, 302, 303, 325, 331, 348, 349, 350, 354, 358, 359, 363, 365, 372, 373, 375, 377, 378, 381, 387, 388, 406, 407, 409, 410, 412, 415, 421, 424, 448, 454, 457, 467, 472, 475, 480, 481, 484, 488, 493, 498, 507, 513]


In [2]:
from bonito_pretrained import BonitoPretrained
checkpoint_path = 'checkpoints_pl/bonito_tl_0/epoch=0-step=4450000.ckpt'
model = BonitoPretrained(pretrained_lr=1e-4, my_layers_lr=1e-3, warmup_steps = 10000).load_from_checkpoint(checkpoint_path)

/home/jovyan/my-conda-envs/RNAmodEnv/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jovyan/my-conda-envs/RNAmodEnv/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")


In [3]:
import numpy as np
def probs_to_bin(probs):
    res = []
    for pos_prob in probs:
        pos_prob = pos_prob[0]
        res.append([1-pos_prob, pos_prob])
    predz = np.array(res)
    return predz

In [4]:
from cleanlab.filter import find_label_issues
from torch import nn
from pathlib import Path

count = 0
logs = []
for x,y,identifier in valid_loader:
    count+=1
    # print('count', count)
    probs = nn.Softmax(dim=0)(model(x)).detach().numpy()
    probs = probs_to_bin(probs)
    identifier['probs'] = probs
    
    identifier_len = len(identifier['label'])
    for i in range(identifier_len):
        new_identifier = {}
        for k,v in identifier.items():
            new_identifier[k] = v[i]
        new_identifier['file'] = Path(new_identifier['file']).stem
        logs.append(new_identifier)
            
    if(count>100):
        break       
        
        
labels = []
preds = []
for ident in logs:
    labels.append(ident['label'].numpy())
    preds.append(ident['probs'])

labels = np.array(labels).astype(np.int32).flatten()



labels[0] = 0 #TODO DELETE, only for evaluationg with all positives  test 




probs = np.array(preds).reshape(-1,2)


        
#TODO run this only once for all predictions
ordered_label_issues = find_label_issues(
    labels=labels,
    pred_probs=probs, 
    # return_indices_ranked_by='self_confidence',
)

for log, is_mislabeled in zip(logs, ordered_label_issues):
    log['is_mislabeled'] = is_mislabeled


[78, 101, 190, 301, 358, 480]
[71, 256, 302, 350, 365, 407]
[164, 303, 359, 424, 488, 493]
[46, 103, 236, 252, 263, 264][9, 159, 276, 295, 381, 387][24, 180, 191, 198, 454, 475][123, 149, 269, 325, 349, 375][10, 17, 117, 177, 247, 410]
[13, 165, 228, 241, 331, 467]
[2, 32, 44, 87, 88, 378]LAST WORKER





[33, 48, 193, 363, 372, 448][58, 105, 377, 409, 412, 457]
[86, 92, 137, 166, 185, 221, 233, 348, 388, 406, 421, 472, 498, 513]
[93, 173, 251, 285, 354, 415][8, 74, 182, 231, 481, 484]


[7, 36, 42, 64, 373, 507]


/home/jovyan/my-conda-envs/RNAmodEnv/lib/python3.8/site-packages/cleanlab/filter.py:740: UserWarning: May not flag all label issues in class: 0, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(


In [6]:
read_ids = np.unique([log['readid'] for log in logs])
results = []
for read_id in read_ids:
    filtered_logs = [log for log in logs if log['readid'] == read_id]
    starts = np.array([log['start'].numpy() for log in filtered_logs])
    assert len(np.unique(starts)) == len(starts)
    label = filtered_logs[0]['label']
    is_mislabeled_map = [log['is_mislabeled'] for log in filtered_logs]
    is_mislabeled_consensus = sum(is_mislabeled_map)/len(is_mislabeled_map)
    results.append({
        'read_id':read_id,
        'label':label,
        'is_mislabeled_consensus':is_mislabeled_consensus,
    })
    
results

[{'read_id': '00014da3-d5ae-4930-898d-8ae795b08e01',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.5454545454545454},
 {'read_id': '00021b92-e9b3-415f-b68e-07ec3653a317',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.5625},
 {'read_id': '0004d7e8-4369-4261-a580-235cb2b47547',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.5555555555555556},
 {'read_id': '00053a16-7190-4e4f-ae20-453ce974f335',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.5777777777777777},
 {'read_id': '00060feb-a1fa-40ec-9a6f-3b46e06a2183',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.72},
 {'read_id': '00095256-22c5-4aaf-8b67-0ce7e8dd5326',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.6521739130434783},
 {'read_id': '0009bcfa-d984-4f8d-abfb-b923a55ca7a7',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.6458333333333334},
 {'read_id': '000e31bb-ad4d-4d24-85e3-a38c75ed15e6',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.5},
 {'read_id': '000ebbee-61cc-4515-b19a-4d2

In [19]:
import pickle

with open('mislabeled_results','wb') as f:
    pickle.dump(results, f)

In [20]:
with open('mislabeled_results','rb') as f:
    results = pickle.load(f)
results

[{'read_id': '0049a707-80e6-4165-9a2a-b5df63c28d6c',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.0},
 {'read_id': '004db83d-a679-485d-b92d-82f2e2424815',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.6666666666666666},
 {'read_id': '005aa046-b1aa-46d8-a122-bc6544453273',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.5877862595419847},
 {'read_id': '005d1a54-6e51-40f9-bdc6-2a3aa364c822',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.5172413793103449},
 {'read_id': '00662f78-77ad-4d19-8737-68cc863d261e',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.6756756756756757},
 {'read_id': '0073891c-1b6e-499a-9757-83ffbc6f2064',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.7195121951219512},
 {'read_id': '0077e468-24fb-4f7e-ac45-d02157c84338',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.5},
 {'read_id': '007c1677-8520-4185-9819-caac6c97eb95',
  'label': tensor(1),
  'is_mislabeled_consensus': 0.6153846153846154},
 {'read_id': '007f5925-1841-4b